#Text Input

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login
import warnings
warnings.filterwarnings("ignore")

def get_medical_response_from_text(text_input):
    """
    Generate a personalized medical analysis based on user-provided text.
    """
    prompt = (
        "Patient query: " + text_input + "\n"
        "Provide a detailed, personalized medical analysis and advice based on the "
        "information provided."
    )
    try:
       # Map model choices to verified model identifiers
        model_mapping = {
            "small": "medalpaca/medalpaca-7b",  # Smaller, easier to run
            "medium": "StanfordAIMI/Med-Flamingo-7B",  # Good balance
            "large": "meditron-7b"  # This might need specific access
        }

        model_name = model_mapping.get(model_choice, "medalpaca/medalpaca-7b")
        print(f"Loading {model_name}...")

        # Check device
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load tokenizer and model with error handling

        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            tokenizer.pad_token = tokenizer.eos_token  # Set pad token

            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if device == "cuda" else torch.float32,
                device_map="auto" if device == "cuda" else None,
                low_cpu_mem_usage=True,
                load_in_8bit=True if device == "cuda" else False
            )

        except Exception as e:
            print(f"Error loading {model_name}: {e}")
            print("Falling back to smaller model...")
            return run_medical_model(user_input, "small")

        # Create pipeline
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=0 if device == "cuda" else -1,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32
        )
        # Generate response
        print("Medical Recommendation :")
        outputs = pipe(
            prompt,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

        # Extract response
        full_response = outputs[0]['generated_text']
        response = full_response.split("### Assistant Response:")[-1].strip()

        return response

    except Exception as e:
        return f"Error: {str(e)}"

def main():
    print("Enter your medical query (text input):")
    user_input = input(">> ")
    result = get_medical_response_from_text(user_input)
    print("\nMedical Recommendation:")
    print(result)

if __name__ == "__main__":
    main()


Enter your medical query (text input):
>> As I have asthma, what are the food products that l need to avoid in order to keep my balance keep it short.

Medical Recommendation:
As an asthma patient, it's important to be mindful of certain foods that may trigger or worsen your symptoms. Some common food triggers for asthma include:

1. Dairy products: Some people with asthma may be sensitive to dairy products, which can cause inflammation and increase mucus production in the airways.

2. Processed and fried foods: These foods are often high in trans fats and other unhealthy ingredients that can promote inflammation in the body and worsen asthma symptoms.

3. Excessive salt: High sodium intake can lead to fluid retention and may worsen breathing difficulties in asthma patients.

4. Sulfites and preservatives: Foods and drinks that contain sulfites and other preservatives can trigger asthma symptoms in some individuals.

5. Foods high in histamines: Histamine-rich foods like fermented dair

#PDF/Document



In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login
import warnings
warnings.filterwarnings("ignore")
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF document using PyPDF2.
    """
    text_content = ""
    try:
        with open(pdf_path, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text_content += page_text + "\n"
    except Exception as e:
        print(f"Error reading PDF file: {str(e)}")
    return text_content

def get_medical_response_from_pdf(text_input):
    """
    Uses the Meditron 7B to generate a medical analysis based on extracted PDF text.
    """
    prompt = (
        "Extracted medical document content: " + text_input + "\n"
        "Provide a detailed analysis and personalized recommendations based on the content."
    )
    try:
       # Map model choices to verified model identifiers
        model_mapping = {
            "small": "medalpaca/medalpaca-7b",  # Smaller, easier to run
            "medium": "StanfordAIMI/Med-Flamingo-7B",  # Good balance
            "large": "meditron-7b"  # This might need specific access
        }

        model_name = model_mapping.get(model_choice, "medalpaca/medalpaca-7b")
        print(f"Loading {model_name}...")

        # Check device
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load tokenizer and model with error handling

        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            tokenizer.pad_token = tokenizer.eos_token  # Set pad token

            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if device == "cuda" else torch.float32,
                device_map="auto" if device == "cuda" else None,
                low_cpu_mem_usage=True,
                load_in_8bit=True if device == "cuda" else False
            )

        except Exception as e:
            print(f"Error loading {model_name}: {e}")
            print("Falling back to smaller model...")
            return run_medical_model(user_input, "small")

        # Generate response
        print("Medical Recommendation :")
        outputs = pipe(
            prompt,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

        # Extract response
        full_response = outputs[0]['generated_text']
        response = full_response.split("### Assistant Response:")[-1].strip()

        return response

    except Exception as e:
        return f"Error: {str(e)}"

def main():
    pdf_file_path = input("Enter the path to your medical PDF document: ")
    extracted_text = extract_text_from_pdf(pdf_file_path)
    if extracted_text:
        print("\nExtracted Text from PDF (first 500 characters):")
        print(extracted_text[:500] + "...\n")
        result = get_medical_response_from_pdf(extracted_text)
        print("Medical Analysis from PDF:")
        print(result)
    else:
        print("No text extracted from the PDF.")

if __name__ == "__main__":
    main()

Enter the path to your medical PDF document: /content/genome-report-example.pdf

Extracted Text from PDF (first 500 characters):
_______________________________ _____________________________________________________________________________________________________________________________ __________________________  
Laboratory for Molecular Medicine                                                                                                                                                                 Matthew S. Lebo, Ph.D., FACMG, Director  
https://www.massgeneralbrigham.org/en/research -and-innovation/centers -and-p...

Medical Analysis from PDF:
The extracted medical document provides detailed genetic information regarding monogenic disease findings, risk alleles, carrier status variants, and pharmacogenomic associations for an individual. 

1. Monogenic Disease Findings: The test did not identify any variants with the potential to cause monogenic disease in the individual.

2. 

#Vision

In [ ]:
!pip install openai pymupdf opencv-python pytesseract

In [ ]:
import os
import fitz  # PyMuPDF for PDF processing
import base64  # For encoding images
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def setup_meditron_model():
    try:
        model_name = "medalpaca/medalpaca-7b"
        tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)
        tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            low_cpu_mem_usage=True
        )

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=0 if torch.cuda.is_available() else -1
        )

        return pipe
    except Exception as e:
        raise Exception(f"Error loading Meditron model: {str(e)}")

def extract_mri_text_from_pdf(pdf_path):
    try:
        if not os.path.exists(pdf_path):
            return "Error: File not found."

        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                page_text = page.get_text("text")
                if "MRI" in page_text or "magnetic resonance imaging" in page_text.lower():
                    text += page_text + "\n"

        return text.strip() if text.strip() else "Error: No MRI-related text detected in the PDF."

    except Exception as e:
        return f"Error extracting text from PDF: {str(e)}"

def encode_image(image_path):
    try:
        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")
    except Exception as e:
        return f"Error encoding image: {str(e)}"

def analyze_with_meditron(pipe, prompt, max_tokens=500):
    try:
        formatted_prompt = f"""### Medical Analysis Task:
You are a medical expert analyzing MRI reports and scans. Provide detailed, professional medical insights.

### Input:
{prompt}

### Medical Analysis:
"""
        response = pipe(
            formatted_prompt,
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=pipe.tokenizer.eos_token_id
        )

        full_text = response[0]['generated_text']
        analysis = full_text.split("### Medical Analysis:")[-1].strip()

        return analysis

    except Exception as e:
        return f"Error generating analysis: {str(e)}"

def analyze_medical_document(file_path):
    try:
        if not os.path.exists(file_path):
            return "Error: The file does not exist. Check the path."

        pipe = setup_meditron_model()

        if file_path.lower().endswith((".png", ".jpg", ".jpeg")):
            image_data = encode_image(file_path)
            if "Error" in image_data:
                return image_data

            prompt = f"""MRI Image Analysis Request:

                I have an MRI scan image (base64 encoded: {image_data[:100]}...). Please provide detailed analysis including:
                1. General interpretation guidelines for this type of scan
                2. Common findings in similar MRI scans
                3. Potential clinical significance
                4. Recommended next steps
                """
            return analyze_with_meditron(pipe, prompt, max_tokens=600)

        elif file_path.lower().endswith(".pdf"):
            extracted_text = extract_mri_text_from_pdf(file_path)
            if "Error" in extracted_text:
                return extracted_text

            prompt = f"""Extracted MRI text from document:
{extracted_text}

Analyze this MRI report and provide a detailed medical recommendation, including any potential conditions, observations, and next steps.
"""
            return analyze_with_meditron(pipe, prompt, max_tokens=700)

        else:
            return "Unsupported file type. Please provide a JPG, PNG, or PDF."

    except Exception as e:
        return f"Error processing the document: {str(e)}"

def main():
    file_path = input("Enter the path to your medical file (image or PDF): ").strip()
    result = analyze_medical_document(file_path)
    print("\n📄 Medical Analysis Result:")
    print(result)

if __name__ == "__main__":
    main()

Enter the path to your medical file (image or PDF): /content/mrinew.jpeg

📄 **Medical Analysis Result:**
The provided MRI scan images include sections of both the knee and the lower extremity (possibly the ankle), presented in various planes—primarily the sagittal and coronal planes, which are common orientations used for detailed evaluation of joint structures.

### Analysis:

1. **Knee Analysis (Sagittal and Coronal Views)**
   - **Structures Visible**: Femur, tibia, fibula, knee joint space, menisci, and surrounding soft tissues.
   - **Menisci**: The menisci appear to be intact without evident degeneration or tearing. Both medial and lateral menisci display normal shape and signal intensity.
   - **Ligaments**: Cruciate ligaments (ACL and PCL) and collateral ligaments (MCL and LCL) are not clearly visible in every image but where visible, they do not show signs of rupture or significant alteration.
   - **Cartilage**: The articular cartilage covering the ends of the femur and tibia